In [1]:
# importing necessary modules.
import pandas as pd
import config as cfg
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

In [2]:
# creating a connection to an aws rds postgres cloud database.
rds_connection_string = f'postgres:{cfg.password}@test-db.cy2enoewwvsi.us-east-2.rds.amazonaws.com:5432/stocks_db'
engine = create_engine(f'postgres://{rds_connection_string}')

In [3]:
# checking out the table names in the database.
engine.table_names()

['russell_2000', 'sandp', 'sandp_russell']

In [4]:
# checking out the SandP_500 table.
sandp = pd.read_sql_query('SELECT * FROM sandp', con=engine)
sandp.head()

,symbol,name,price_to_bookvalue,price_to_book,price_to_sales,price_to_earnings,receivables_turnover,price_to_free_cash_flow,price_to_operating_cash_flow,enterprise_value_multiple,gross_profit_margin,pretax_profit_margin,net_profit_margin,tax_rate,return_on_assets,return_on_equity,return_on_capital_employed,receivables_turnover_1,payables_turnover,inventory_turnover,fixed_asset_turnover,asset_turnover,current_ratio,quick_ratio,cash_ratio,days_of_sales_outstanding,days_of_inventory_outstanding,operating_cycle,days_of_payables_outstanding,cash_conversion_cycle,debt_ratio,debt_to_equity_ratio,longterm_debt_to_capitalization,total_debt_to_capitalization,interest_coverage,cash_flow_to_debt,company_equity_multiplier,operating_cash_flow_per_share,free_cash_flow_per_share,cash_per_share,payout_ratio,receivables_turnover_2,operating_cash_flow_to_sales,free_cash_flow_to_operating_cash_flow,cash_flow_coverage_ratio,short_term_coverage_ratio,capital_expenditure_coverage_ratio,dividend_capex_coverage_ratio,dividend_payout_ratio,gross_profit_growth,ebit_growth,operating_income_growth,net_income_growth,eps_growth,eps_diluted_growth,weighted_average_share_growth,weighted_averages_shares_diluted_growth,dividents_per_share_growth,operating_cash_flow_growth,free_cash_flow_growth,ten_year_revenue_growth_per_share,five_year_revenue_growth_per_share,three_year_revenue_growth_per_share,ten_year_operating_cf_growth_per_share,five_year_operating_cf_growth_per_share,three_year_operating_cf_growth_per_share,ten_year_net_income_growth_per_share,five_year_net_income_growth_per_share,three_year_net_income_growth_per_share,ten_year_shareholders_equity_growth_per_share,five_year_shareholders_equity_growth_per_share,three_year_shareholders_equity_growth_per_share,ten_year_dividend_growth_per_share,five_year_dividend_growth_per_share,three_year_dividend_growth_per_share,receivables_growh,inventory_growth,asset_growth,book_value_per_share_growth,debt_growth,randd_expense_growth,sganda_expense_growth,percent_return_on_investment,above_below_sandp_return,market_cap,industry,sector
0,A,"Agilent Technologies, Inc.",2.5517,5.17165,1.48130,6.30885,7.0014,18.18070,13.57995,16.746223,0.524850,0.057228,0.059362,2.720066,0.0799,0.09975,0.0513,7.0014,3.85290,8.0338,5.429030,0.575072,2.7640,2.894055,2.481972,-97.99795,53.36445,-99.0,29.23140,-99.0,0.38115,1.15130,0.470496,0.535164,4.6439,0.167487,3.020614,1.62420,1.2763,11.89585,-99.0000,7.0014,0.111470,0.764976,0.167487,204.239174,4.755422,4.755422,-99.000000,-0.00225,3.19800,5.05085,11.00990,10.92090,10.75375,-0.02195,-0.0236,-99.00000,0.14975,0.30285,-0.03010,0.08565,0.04200,0.04210,-0.00300,-0.06355,0.0079,0.12195,0.03385,-0.02165,0.01000,-0.00400,-99.00000,-99.00000,-99.0000,0.11660,0.07580,0.18005,0.15185,0.31885,-0.0674,0.01880,215.407785,1,7.462700e+09,Medical Diagnostics & Research,Healthcare
1,AAL,"American Airlines Group, Inc.",-99.0000,-99.00000,0.11565,-99.00000,27.3348,-99.00000,2.20925,61.812855,0.495744,-0.018258,-0.047475,-99.000000,-0.0689,0.27010,-0.0152,27.3348,5.02710,37.6691,1.378462,0.833326,0.8190,0.687756,0.074855,-19.81890,13.11230,-99.0,19.26545,-99.0,0.45185,-3.08765,1.617521,1.486838,-0.4848,0.095474,-99.000000,3.26045,-1.8500,1.84910,-99.0000,27.3348,0.051335,-0.565762,0.095474,0.746942,0.638704,0.638704,-99.000000,0.03935,0.77400,0.88765,0.49305,0.55295,0.55295,0.09755,-99.0000,-99.00000,1.00075,0.16005,-0.06610,-0.12145,-0.13695,-0.15725,-0.09305,-0.27055,-99.0000,-99.00000,-99.00000,-99.00000,-99.00000,-99.00000,-99.00000,-99.00000,-99.0000,-0.04605,0.06370,-0.00170,-0.06235,0.00975,-99.0000,0.01435,492.976589,1,6.133809e+08,Airlines,Industrials
2,AAN,"Aaron's, Inc.",1.5315,1.95610,0.78855,12.37860,27.7757,59.12785,19.92670,0.126805,0.812846,0.103087,0.063665,0.370173,-99.0000,0.12390,-99.0000,27.7757,2.51545,-99.0000,1.896709,1.287786,-99.0000,-99.000000,-99.000000,-99.00000,13.65740,-99.0,39.02640,-99.0,0.03475,0.05235,-99.000000,-99.000000,52.3004,-99.000000,1.511503,1.49430,1.0025,1.1184

In [5]:
# dropping unnecessary columns.
sandp.drop(['symbol', 'name', 'percent_return_on_investment'], axis=1, inplace=True)

In [6]:
sandp.drop(['industry', 'sector'], axis=1, inplace=True)

In [7]:
X = sandp.drop('above_below_sandp_return', axis=1)
target = sandp['above_below_sandp_return']

In [68]:
names = []
for x in X.columns:
    names.append(x)

In [56]:
# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X, target)
rf.score(X, target)

1.0

In [57]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances

array([0.0178298 , 0.01762062, 0.01967006, 0.01138562, 0.01110426,
       0.01200951, 0.01475049, 0.01214905, 0.01216221, 0.01676098,
       0.01426686, 0.01226985, 0.01324637, 0.0154383 , 0.00977244,
       0.01029544, 0.01557081, 0.00970332, 0.02038559, 0.01507497,
       0.01115246, 0.01066363, 0.0108151 , 0.01075389, 0.01110609,
       0.00190103, 0.01808826, 0.00203595, 0.01364066, 0.01064639,
       0.0081196 , 0.00845542, 0.01079808, 0.00826479, 0.01487125,
       0.01752393, 0.0156613 , 0.01922709, 0.00815163, 0.01022816,
       0.01501744, 0.01287769, 0.00855084, 0.00813183, 0.01449172,
       0.02222644, 0.00906281, 0.01500542, 0.01340634, 0.01387396,
       0.01264162, 0.01203196, 0.01232919, 0.0134498 , 0.01131422,
       0.00722603, 0.01707012, 0.01373148, 0.01125234, 0.01167023,
       0.01139632, 0.00851934, 0.0106507 , 0.01155358, 0.00793129,
       0.01080529, 0.01059078, 0.01078046, 0.01240178, 0.01162286,
       0.00578498, 0.00583985, 0.00683833, 0.01148657, 0.00846

In [8]:
# We can sort the features by their importance
sorted(zip(rf.feature_importances_, names), reverse=True)

NameError: name 'rf' is not defined

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, target_train, target_test = train_test_split(X, target, random_state=1, stratify=target)

In [42]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
classifier.fit(X_train, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
print(f'Training Data Score: {classifier.score(X_train, target_train)}')
print(f'Testing Data Score: {classifier.score(X_test, target_test)}')

Training Data Score: 0.7407407407407407
Testing Data Score: 0.48717948717948717


In [45]:
predictions = classifier.predict(X_test)
pd.DataFrame({'Prediction': predictions, 'Actual': target_test}).reset_index(drop=True)

,Prediction,Actual
0,1,0
1,1,0
2,1,0
3,1,1
4,1,0
5,1,0
6,0,0
7,1,0
8,1,0
9,0,1


In [46]:
target_test.value_counts()
target_test.mean()

0.5555555555555556

In [47]:
target_test.value_counts()

1    65
0    52
Name: above_below_sandp_return, dtype: int64